In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t
import moyra as ma
me.mechanics_printing()
%load_ext line_profiler
%load_ext snakeviz

In [2]:

rigid_dofs = 0
bending = 3
in_plane_bend = 0
torsion = 2
inner_dofs = bending + in_plane_bend + torsion
DoFs = rigid_dofs + inner_dofs + 1

# heave and fold angle as degrres of freedom
p = ma.DynamicModelParameters(DoFs)
p.DoFs  = ma.ModelSymbol(value = DoFs,string = 'DoFs')
#gravity vector
p.g  = ma.ModelSymbol(value = 9.81,string = 'g')                     # gravity
p.g_v = ma.ModelMatrix(value =[0,0,-1],length=3,string='g_v') # The stationary point

In [3]:
# inner wing properites
p.rho_b = ma.ModelSymbol(value = 0.3,string = 'rho_b')
p.rho_w = ma.ModelSymbol(value = 7.926,string = 'rho_w')
p.c_b = ma.ModelSymbol(value = 0.3,string = 'c_b')
p.c_w = ma.ModelSymbol(value = 0.132,string = 'c_w')
p.L = ma.ModelSymbol(value = 0.3,string = 'L')
p.c = ma.ModelSymbol(value = 0.3,string = 'c')
p.x_f = ma.ModelSymbol(value = 0.3,string = 'x_f')
p.x_h = ma.ModelSymbol(value = 0.3,string = 'x_h')
p.EI = ma.ModelSymbol(value = 0.3,string = 'EI')
p.GJ = ma.ModelSymbol(value = 0.3,string = 'GJ')

p.m_e = ma.ModelSymbol(value = 0,string = 'm_e')
p.x_e = ma.ModelSymbol(value = 0,string = 'x_e')
p.y_e = ma.ModelSymbol(value = 0,string = 'y_e')

p.y_0,p.x_0 = sym.symbols('y_0,x_0')

In [4]:
# fwt properties
## pendulum mass
p.m_f = ma.ModelSymbol(value = 0.3,string = 'm_f')
p.I_xx = ma.ModelSymbol(value = 0.3,string = 'I_xx')
p.I_yy = ma.ModelSymbol(value = 0.3,string = 'I_yy')
p.I_zz = ma.ModelSymbol(value = 0.3,string = 'I_zz')
p.I_xy = ma.ModelSymbol(value = 0.3,string = 'I_xy')
p.I_xz = ma.ModelSymbol(value = 0.3,string = 'I_xz')
p.I_yz = ma.ModelSymbol(value = 0.3,string = 'I_yz')

# pendulum length
p.L_f = ma.ModelSymbol(value = 0.3,string = 'L_f')
p.x_c = ma.ModelSymbol(value = 0.3,string = 'x_c')
p.y_c = ma.ModelSymbol(value = 0.3,string = 'y_c')

p.Lambda = ma.ModelSymbol(value = 0.3,string = 'Lambda')

In [5]:
## GET INNER WING ELEMENT
# get shape function
q_f = p.q[rigid_dofs:(rigid_dofs+inner_dofs)]
S,tau = ma.elements.FlexiElement.ShapeFunctions_OBM_IBN_TO(bending,in_plane_bend,torsion,q_f,p.y_0,p.x_0,0,1,type='taylor')
u = S*sym.Matrix(q_f) + sym.Matrix([p.x_0,p.y_0,0])
# get reference frame
# inner_wing_frame = ma.frames.HomogenousFrame().Translate(p.q[0],0,p.q[1]).R_y(p.alpha_r).Translate(*u)
# inner_wing_frame = ma.frames.HomogenousFrame().Translate(p.q[0],0,p.q[1]).R_y(p.q[2]).Translate(*u)
# inner_wing_frame = ma.frames.HomogenousFrame().Translate(p.q[0],p.q[1],p.q[2]).R_x(p.q[3]).R_y(p.q[4]).R_z(p.q[5]).Translate(*u)
inner_wing_frame = ma.frames.HomogenousFrame().Translate(*u)
# get flexible element
link2_x__int = (p.x_0,p.x_f-p.c_b*sym.Rational(1,2),p.x_f+p.c_b*sym.Rational(1,2))
link2_y_int = (p.y_0,0,p.L)
main_wing_ele = ma.elements.FlexiElement(p.q,inner_wing_frame,p.rho_b,S,link2_x__int,link2_y_int,0,q_f,p.EI,p.GJ,grav_vec = p.g*p.g_v,simplify = False)
link2_x__int = (p.x_0,0,p.c_w)
link2_y_int = (p.y_0,0,p.L)
main_wing_mass = ma.elements.FlexiElement(p.q,inner_wing_frame,p.rho_w,S,link2_x__int,link2_y_int,0,q_f,0,0,grav_vec = p.g*p.g_v,simplify = False)

## engine mass
engine_frame = inner_wing_frame.subs({p.x_0:p.x_e,p.y_0:p.y_e})
eng_ele = ma.elements.RigidElement.point_mass(p.q,engine_frame,p.m_e,grav_vec = p.g*p.g_v,simplify = False)

In [6]:
## GET FWT ELEMENT

# get angles at end of inner beam
dz_dy = u.subs({p.x_0:0}).diff(p.y_0).subs({p.y_0:p.L})
dz_dx = u.subs({p.y_0:p.L}).diff(p.x_0).subs({p.x_0:0})


# inner_tip_frame= inner_wing_frame.subs({p.y_0:p.L,p.x_0:0}).R_y(dz_dx[2]/dz_dx[0]).R_x(dz_dy[2]/dz_dy[1])
inner_tip_frame= inner_wing_frame.subs({p.y_0:p.L,p.x_0:0}).R_x(dz_dy[2]/dz_dy[1]).simplify()
hinge_frame = inner_tip_frame.R_z(p.Lambda).simplify()
fwt_base_frame = hinge_frame.R_x(-p.q[-1]).R_z(-p.Lambda).simplify()
fwt_frame = fwt_base_frame.Translate(p.x_c,p.y_c,0)

In [7]:
#Create Elemnts
# M = ma.elements.MassMatrix(p.m_f,p.I_xx,p.I_yy,p.I_zz,p.I_xy,p.I_xz,p.I_yz)
M = ma.elements.MassMatrix(p.m_f,p.I_xx,p.I_yy,p.I_zz)
# M = ma.elements.MassMatrix(p.m_f)
fwt_ele = ma.elements.RigidElement(p.q,fwt_frame,M,grav_vec = p.g*p.g_v, simplify = False)

In [8]:
%%time
# %%snakeviz -t

sm = ma.SymbolicModel.FromElementsAndForces(p.q,[main_wing_mass,main_wing_ele,fwt_ele,eng_ele])
# %lprun -f ma.SymbolicModel.FromElementsAndForces ma.SymbolicModel.FromElementsAndForces(p.q,[main_wing_ele,fwt_ele])

Generating EoM for Element 1 out of 4 - default:FlexiElement
Generating EoM for Element 2 out of 4 - default:FlexiElement
Generating EoM for Element 3 out of 4 - default:RigidElement
Generating EoM for Element 4 out of 4 - default:RigidElement
CPU times: total: 1min 59s
Wall time: 1min 59s


In [9]:
funcs = []
funcs.append(('get_pos_inner_wing',inner_wing_frame.transform_point([0]*3)))
funcs.append(('get_pos_fwt',fwt_frame.transform_point([p.x_0,p.y_0,0])))

In [10]:
fwt_normal = fwt_frame.transform_vector([0,0,1])
fwt_hinge_vector = fwt_frame.transform_vector([sym.cos(p.Lambda),sym.sin(p.Lambda),0])
surf = inner_wing_frame.transform_point([0]*3)
v = ma.Wedge(surf.diff(p.x_0))*surf.diff(p.y_0)
hinge_normal = sym.simplify(v.subs({p.x_0:p.x_h,p.y_0:p.L}))
surf = inner_wing_frame.transform_point([0]*3)
funcs.append(('get_surf_x',surf.subs({p.x_0:1})-surf.subs({p.x_0:0})))
funcs.append(('get_surf_z',sym.simplify(v)))
funcs.append(('get_fold',p.q[-1]))
funcs.append(('get_fold_dt',p.qd[-1]))
funcs.append(('get_hinge_normal',hinge_normal))
funcs.append(('get_fwt_normal',fwt_normal))
funcs.append(('get_fwt_hinge_vector',fwt_hinge_vector))

In [11]:
#create function for an arbitary force of FWT
F_x,F_y,F_z,M_y = sym.symbols('F_x,F_y,F_z,M_y')
F = ma.forces.ExternalForce.body_force(p.q,fwt_frame.Translate(p.x_0,p.y_0,0),F_x,F_y,F_z,0,M_y,0,False)
funcs.append(('get_fwt_Q',F.Q()))
#create function for an arbitary force on Main Wing
F = ma.forces.ExternalForce.body_force(p.q,inner_wing_frame,F_x,F_y,F_z,0,M_y,0,False)
funcs.append(('get_main_Q',F.Q()))

In [12]:
#create function for A of each frame
funcs.append(('get_fwt_A',fwt_frame.A))
funcs.append(('get_main_A',inner_wing_frame.A))

In [13]:
#create function for Body Velocity on FWT and Main wing
fwt_aero_frame = fwt_frame.Translate(p.x_0,p.y_0,0)
funcs.append(('get_fwt_V_b',fwt_aero_frame.A.T*fwt_aero_frame.R.diff(t)))
# funcs.append(('get_fwt_V_b',fwt_ele.frame.transform_point([p.x_0,p.y_0,0]).diff(t)))
funcs.append(('get_main_V_b',inner_wing_frame.A.T*inner_wing_frame.R.diff(t)))

In [14]:
%timeit -r 1 -n 1
base_dir = "C:\\git\\WT_MBD_Model"
class_name = "Flexi_wing_RC"
base_class = "mbd.BaseRC"
sm.to_matlab_class(p,base_dir,class_name,base_class,additional_funcs=funcs)

Unknown variable y_0 found in function get_pos_inner_wing. It will be added to the function signature.
Unknown variable x_0 found in function get_pos_inner_wing. It will be added to the function signature.
Unknown variable x_0 found in function get_pos_fwt. It will be added to the function signature.
Unknown variable y_0 found in function get_pos_fwt. It will be added to the function signature.
Unknown variable y_0 found in function get_surf_x. It will be added to the function signature.
Unknown variable y_0 found in function get_surf_z. It will be added to the function signature.
Unknown variable x_0 found in function get_surf_z. It will be added to the function signature.
Unknown variable F_z found in function get_fwt_Q. It will be added to the function signature.
Unknown variable F_x found in function get_fwt_Q. It will be added to the function signature.
Unknown variable M_y found in function get_fwt_Q. It will be added to the function signature.
Unknown variable x_0 found in funct